In [1]:
import h5py
import tensorflow as tf
import numpy as np
from numpy import f2py

import time

import eos
import eos_t

# TensorFlow

## Vectorization with gmlt_spec_od_grid

### outline
- arr has shape (x,y,z)
- reshape arr to (x*y, z)
- vectorized_map(fn, arr) returns arr2, shape (x*y, z)
    - here, fn is gmlt_spec_od_pwc_exact
- reshape arr2 to (x,y,z)
- create od_pwc_exact2: only arg is a tuple of the 3 grids, while snapshot-specific args (e.g. redshift) are specified outside the function
    - define this within gmlt_spec_od_grid

In [2]:
def fn(a, factor):
    return a*factor

fact = 2
def fn2(a):
    return fn(a, fact)

# testing tf.reshape

arr = tf.constant(np.arange(24).reshape(2,3,4))
#print(arr)

# we want to reshape arr to [[0,1,2,3], ..., [20,21,22,23]]
arr = tf.reshape(arr, (6, 4))

# arr2 = tf.vectorized_map(fn2, arr)
arr2 = tf.map_fn(fn2, arr, fn_output_signature=tf.int64)
print(arr2)
print(tf.reshape(arr2, (2,3,4)))

tf.Tensor(
[[ 0  2  4  6]
 [ 8 10 12 14]
 [16 18 20 22]
 [24 26 28 30]
 [32 34 36 38]
 [40 42 44 46]], shape=(6, 4), dtype=int64)
tf.Tensor(
[[[ 0  2  4  6]
  [ 8 10 12 14]
  [16 18 20 22]]

 [[24 26 28 30]
  [32 34 36 38]
  [40 42 44 46]]], shape=(2, 3, 4), dtype=int64)


In [3]:
@tf.function()
def outer_product(a):
    return tf.tensordot(a, a, 0)

batch_size = 10
a = tf.ones((batch_size, 4, 4))
c = tf.vectorized_map(outer_product, a)
c.shape

TensorShape([10, 4, 4, 4, 4])

## Vectorization with nyx_eos

In [4]:
eos_obj = eos.EOS_at_z(2.99)

# nyx_eos can accept tensors of different dtypes without errors
t1 = tf.Variable([2], dtype=tf.int32)
t2 = tf.constant([2], dtype=tf.float64)
eos_obj.nyx_eos_vec((t1, t2))

9.087522466956245e+23

In [12]:
x = tf.Variable(np.arange(100).reshape(10,10), dtype=tf.double)
y = tf.Variable(np.arange(100).reshape(10,10), dtype=tf.double)

size = tf.size(x).numpy()
#elems=(x,y)
elems = (tf.reshape(x, [size]), tf.reshape(y, [size]))

# vectorized_map throws a "[Tensor] can't be converted to double" error
#tf.vectorized_map(eos_obj.nyx_eos_vec, elems)
tf.map_fn(eos_obj.nyx_eos_vec, elems, fn_output_signature=tf.float64)

<tf.Tensor: shape=(100,), dtype=float64, numpy=
array([0.00000000e+00, 4.54376123e+23, 9.08752247e+23, 1.36312837e+24,
       1.81750449e+24, 2.27188062e+24, 2.72625674e+24, 3.18063286e+24,
       3.63500899e+24, 4.08938511e+24, 4.54376123e+24, 4.99813736e+24,
       5.45251348e+24, 5.90688960e+24, 6.36126573e+24, 6.81564185e+24,
       7.27001797e+24, 7.72439410e+24, 8.17877022e+24, 8.63314634e+24,
       9.08752247e+24, 9.54189859e+24, 9.99627471e+24, 1.04506508e+25,
       1.09050270e+25, 1.13594031e+25, 1.18137792e+25, 1.22681553e+25,
       1.27225315e+25, 1.31769076e+25, 1.36312837e+25, 1.40856598e+25,
       1.45400359e+25, 1.49944121e+25, 1.54487882e+25, 1.59031643e+25,
       1.63575404e+25, 1.68119166e+25, 1.72662927e+25, 1.77206688e+25,
       1.81750449e+25, 1.86294211e+25, 1.90837972e+25, 1.95381733e+25,
       1.99925494e+25, 2.04469256e+25, 2.09013017e+25, 2.13556778e+25,
       2.18100539e+25, 2.22644300e+25, 2.27188062e+25, 2.31731823e+25,
       2.36275584e+25, 2.4081

## Vectorization example

In [54]:
# a function with specified parameter types can accept tf.Tensors
def fn2(a: float, b: float):
    return a + b

In [29]:
# elems is a set of tensors, c is a number
def fn(elems, c=3):
    tf.compat.v1.enable_eager_execution()
    print('eager:', tf.executing_eagerly()) # still False

    # convert_to_tensor doesn't fix it
    a = tf.convert_to_tensor(elems[0], dtype=tf.float32, name='test')
    b = elems[1]
    
    print('a:', a)
    print('type(a):', type(a))
    print('eager:', tf.executing_eagerly())
    
    # nyx_eos does NOT accept tf.Tensors
    # (when using vectorized_map, a is a tf.Tensor)
    print(eos_obj.nyx_eos((3,a)))
    
    #print(fn2(a,b))
    
    return a + b + c

Getting the value of a tf.Tensor: https://stackoverflow.com/questions/33633370/how-to-print-the-value-of-a-tensor-object-in-tensorflow

In [30]:
x = tf.constant(np.arange(100).reshape(10,10), dtype=tf.float64)
y = tf.constant(np.arange(100).reshape(10,10), dtype=tf.float64)
elems = (x,y)

start = time.time()
print('eager:', tf.executing_eagerly())
out = tf.vectorized_map(fn, elems=elems)

#time_stats(time.time() - start, 10)

eager: True
eager: False


ValueError: in user code:

    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:188 f  *
        iters,
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:248 _pfor_impl  **
        loop_fn_outputs = loop_fn(loop_var)
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:472 loop_fn
        return fn(gathered_elems)
    <ipython-input-29-80fa8ad4df20>:6 fn
        a = tf.convert_to_tensor(elems[0], dtype=tf.float32, name='test')
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:1405 convert_to_tensor_v2_with_dispatch
        value, dtype=dty

In [8]:
out

<tf.Tensor: shape=(10, 10), dtype=float64, numpy=
array([[  3.,   5.,   7.,   9.,  11.,  13.,  15.,  17.,  19.,  21.],
       [ 23.,  25.,  27.,  29.,  31.,  33.,  35.,  37.,  39.,  41.],
       [ 43.,  45.,  47.,  49.,  51.,  53.,  55.,  57.,  59.,  61.],
       [ 63.,  65.,  67.,  69.,  71.,  73.,  75.,  77.,  79.,  81.],
       [ 83.,  85.,  87.,  89.,  91.,  93.,  95.,  97.,  99., 101.],
       [103., 105., 107., 109., 111., 113., 115., 117., 119., 121.],
       [123., 125., 127., 129., 131., 133., 135., 137., 139., 141.],
       [143., 145., 147., 149., 151., 153., 155., 157., 159., 161.],
       [163., 165., 167., 169., 171., 173., 175., 177., 179., 181.],
       [183., 185., 187., 189., 191., 193., 195., 197., 199., 201.]])>

## Assigning an element to an existing tensor

In [7]:
x = tf.zeros([3,3,3])
#tf.tensor_scatter_nd_add?

In [6]:
skewer = tf.zeros([3])
for i in range(3):
    for j in range(3):
        x = tf.tensor_scatter_nd_add(x, [[i,j]], [skewer])
        skewer = skewer + 1
x

<tf.Tensor: shape=(3, 3, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [4., 4., 4.],
        [5., 5., 5.]],

       [[6., 6., 6.],
        [7., 7., 7.],
        [8., 8., 8.]]], dtype=float32)>

In [12]:
x = tf.zeros([3])
tf.tensor_scatter_nd_add(x, [[1]], [3])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 3., 0.], dtype=float32)>

## Testing speed of tensor access

It takes too long to access a tensor's entries one-by-one; I need to avoid this when I'm passing fields into the EOS or optical depth routines

In [7]:
def time_stats(duration, n):
    rate = duration / n**2
    print("Duration:", np.round(duration, 4))
    print("Hours needed to access 1024^3 entries:", 
      np.round(rate * 1024**3 / 3600, 4))

In [61]:
n = 5
field = tf.zeros((n,n))
total = 0

start = time.time()

for i in range(n):
    for j in range(n):
        total += field[i,j]

duration = time.time() - start
time_stats(duration, n)

Duration: 0.0067
Hours needed to access 1024^3 entries: 79.4709


### Nested loops

In [51]:
count = 15
n = 10
field = tf.zeros((n,n))

start = time.time()

# outer loop
i = tf.constant(0)
condition1 = lambda i, count: tf.less(i, n)

def body1(i, r): # index i and result r
    # inner loop
    j = tf.constant(0)
    condition2 = lambda i,j,r: tf.less(j, n)
    
    def body2(i, j, r):
        r += field[i,j]
        return i, tf.add(j, 1), r
    
    i, j, r = tf.while_loop(condition2, body2, loop_vars=[i,j, r])
    
    # increment r
    return tf.add(i, 1), r

# do the loop:
r = tf.while_loop(condition1, body1, [i, count])
print(r)

duration = time.time() - start
time_stats(duration, n)

(<tf.Tensor: shape=(), dtype=int32, numpy=10>, <tf.Tensor: shape=(), dtype=float32, numpy=15.0>)
Duration: 0.0435
Hours needed to access 1024^3 entries: 129.814


## Testing tf gradients

In [13]:
tf.math.pow(z, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [11]:
z = tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(z) # without this line, grad is None
    a = tf.divide(1, z+1)
    print('a:', a)
    
grad = tape.gradient(a, z)
print('grad:', grad)

a: tf.Tensor(0.25, shape=(), dtype=float32)
grad: tf.Tensor(-0.0625, shape=(), dtype=float32)


# Wrapping Fortran functions to Python

[Three ways to wrap](https://numpy.org/doc/stable/f2py/f2py.getting-started.html)

- Quick way (this requires Python 3.7 and NumPy 1.18): `python3 -m numpy.f2py -c eos-t.f90 -m eos_t`


In [3]:
from platform import python_version
print(python_version())

3.7.10


In [5]:
eos_t.atomic_rates.tabulate_rates()
print(eos_t.atomic_rates.alphahp)

[1.26271073e-10 1.25558019e-10 1.24848791e-10 ... 6.57445059e-19
 6.47458444e-19 6.37622769e-19]


# Load in the data

In [1]:
filename = "../../../../../cscratch1/sd/jupiter/sim2_z3_FGPA_cgs.h5"

#snap = h5py.File(filename,'r')

In [5]:
name = 'aux_fields'

snap.keys()

if name in snap:
    print('ok')

ok


## Testing classes

In [2]:
import snapshot
import time

snap = snapshot.Snapshot(filename)

In [3]:
# test snapshot methods
start = time.time()

temp = snap.read_field('/native_fields/temperature')
rhob = snap.read_field('/native_fields/baryon_density')
print(temp.shape)
print(temp.size)
print('Time:', time.time() - start)

[1024 1024 1024]
[21.09375 21.09375 21.09375]
Time: 11.72275996208191


In [3]:
# test read_field2 (reading in n^3 grids)
start = time.time()
shape = [5,5,5]

temp2 = snap.read_field2('/native_fields/temperature', shape)
print(temp2.shape)
print(temp2.size)
print('Time:', time.time() - start)

[5, 5, 5]
tf.Tensor([0.10299683 0.10299683 0.10299683], shape=(3,), dtype=float64)
Time: 17.140169620513916


### universe

In [3]:
u = snap.universe
chi = 10
z = 1
u.chi_to_proper_cgs(chi, z)

<tf.Tensor: shape=(), dtype=float64, numpy=2.2856888888888885e+25>

In [4]:
u.h

<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=0.675>

### eos

In [5]:
import eos

eos_obj = eos.EOS_at_z(snap.z)
# returns nhi
eos_obj.nyx_eos(rhob.field[0,0,0] * 1e-29, temp.field[0,0,0])

9.275646225472467e-13

## Metadata

In [5]:
shape = snap['domain'].attrs['shape']
size = snap['domain'].attrs['size']

z = snap['universe'].attrs['redshift']
omega_b = snap['universe'].attrs['omega_b']
omega_m = snap['universe'].attrs['omega_m']
omega_l = snap['universe'].attrs['omega_l']
h = snap['universe'].attrs['hubble']

scale_factor = 1.0 / (1.0 + z)

In [6]:
print(shape)
print(size)

[1024 1024 1024]
[21.09375 21.09375 21.09375]


In [5]:
snap.close()